This scripts depends on the generatation of envelope features that needs to be generated first.

In [1]:
#filename where you want to store the audio features
filename = 'N95_Audio_Features.pickle'

In [3]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
import latexify as lt
import os
#from thinkdsp import read_json
import scipy
import speechpy
#LOSO Validation
loo = LeaveOneOut()
import tsfel
#for filter
from scipy.signal import butter
from scipy.signal import lfilter
from scipy.signal import freqz;
import pickle

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='highpass', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y;

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
#the file where all data is arranged.
data = pd.read_csv("data_FVC_N95.csv")
print("Total Data Points = ",len(data))
data.head(25)

Total Data Points =  24


,Unnamed: 0,Unnamed: 0.1,Name,Filename,rPEF,rFEV1,rFVC,rRatio,gPEF,gFEV1,gFVC,gRatio
0,0,NaN,Maithili01,Maithili01.FVC_N95.2b66msvi.wav,2.41,0.15,3.45,4.347826,4.39,2.12,2.86,74.125874
1,1,NaN,Rahul01,Rahul01.FVC_N95.2b8ki4jm.wav,1.99,0.10,1.74,5.747126,7.00,3.43,3.43,100.000000
2,2,NaN,ShikhaD01,ShikhaD01.FVC_N95.2b6bnd0l.wav,3.22,0.16,4.85,3.298969,4.50,1.92,2.07,92.753623
3,3,NaN,Vignesh01,Vignesh01.FVC_N95.2bb8mr8l.wav,2.44,0.16,3.37,4.747774,7.12,3.16,3.28,96.341463
4,4,NaN,Yash01,Yash01.FVC_N95.2b8n0616.wav,1.98,0.12,2.02,5.940594,7.81,2.96,2.96,100.000000
5,5,NaN,Sandra01,Sandra01.FVC_N95.2bda6hnu.wav,2.35,0.09,2.46,3.658537,4.07,2.59,2.67,97.003745
6,6,NaN,Hatim01,Hatim01.FVC_N95.2be0sr3s.wav,2.43,0.04,3.75,1.066667,2.52,0.61,2.73,22.344322
7,7,NaN,Maithili02,Maithili02.FVC_N95.2b66lj8n.wav,2.84,0.19,3.56,5.337079,4.39,2.12,2.86,74.125874
8,8,NaN,Rahul02,Rahul02.FVC_N95.2b8kj7lr.wav,0.77,0.04,0.95,4.210526,7.00,3.43,3.43,100.000000
9,9,NaN,Kaushal01,Kaushal01.FVC_N95.2bldi9ts.wav,1.61,0.05,2.06,2.427184,8.42,3.36,3.60,93.333333


In [5]:
#load time series feature of the envelope
    
with open('ts-fel-N95-Envelope.pickle', 'rb') as handle:
    ts_features_envelope = pickle.load(handle)

Audio_Features = {}
    
def get_features(filepath, lungParam):
    
    #use the below line if the data is in json
    #wave = read_json(filepath=filepath)
    
    #use the below line if the data is in wav
    ys, fs = librosa.load(filepath)
    
    if lungParam == 'FEV1':
        #print("FEV1 is the param. Audio signal will be clipped to peak only")
        peakIndex = np.where(ys == ys.max())[0][0]
        ys = ys[0:peakIndex+1];

    ys = speechpy.processing.preemphasis(ys, cof=0.98)
    
    
    #staching frames
    frames = speechpy.processing.stack_frames(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01, filter=lambda x: np.ones((x,)),zero_padding=True)

    power_spectrum = np.mean(speechpy.processing.power_spectrum(frames, fft_points=512), axis=0)
    
    
    mfcc = speechpy.feature.mfcc(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
             num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    mfcc_cmvn = np.mean(speechpy.processing.cmvnw(mfcc,win_size=301,variance_normalization=True), axis=0)
    
    logenergy = speechpy.feature.lmfe(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
             num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    
    logenergy = np.mean(logenergy, axis=0)
    
    mfe = speechpy.feature.mfe(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
            num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    mfe = np.mean(mfe[0],axis=0)
    
    
    #get the features of the HT envelope of audio
    stat_features_envelope = ts_features_envelope[filepath.split('/')[-1]]
    
    
    cfg = tsfel.get_features_by_domain()
    stat_features = np.array(tsfel.time_series_features_extractor(cfg, ys, fs=16000, window=500))[0]
    
    #fc_coefficients = np.mean(librosa.feature.mfcc(y=ys, n_mfcc=13, sr=16000),axis=1)
    #print(mfc_coefficients.shape)
    
    
    melspectogram = np.mean(librosa.feature.melspectrogram(y=ys, sr=16000, n_mels=64, fmax=8000),axis=1)
    #print(melspectogram.shape)
    
    stft_spectrogram=np.abs(librosa.stft(ys))
    chromagram=np.mean(librosa.feature.chroma_stft(S=stft_spectrogram, sr=16000),axis=1)
    
    
    S,phase = librosa.magphase(librosa.stft(ys))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(S=S, sr=fs), axis=1) 
    zcr = np.mean(librosa.feature.zero_crossing_rate(ys)[0])
    
    
    feature_matrix_r = np.hstack((stat_features,stat_features_envelope,mfe, logenergy, mfcc_cmvn,power_spectrum,
                                  melspectogram,chromagram,spectral_rolloff,zcr))
    
    Audio_Features[filepath.split('/')[-1]] = feature_matrix_r;
    
    
    return feature_matrix_r


def load_data(lungParam, sex, LH, file):
    '''
    feature: it is either 'FEV1', 'FVC' or 'PEF'
    '''
    data = pd.read_csv(file)
    
    if sex == 'M' or sex == 'F':
        data = data[data['Sex'] == sex]
    if LH == 'Y' or LH == 'N':
        data = data[data['LH'] == LH]
    
    
    #get the ground truth
    y=  np.array([i for i in data['g'+lungParam]])
    
    #prepare to store a lot of features in X
    X=[]
    count = 0
    for file in data['Filename']:
        features = get_features("SpiroMask-July2021/SmartMaskData/FVC/N95/"+file, lungParam)
        #print("Size of features = ",features.shape)
        X.append(features)
        count += 1
    X =  np.array(X)
    
    
    #add the estiamted lung param as a feature
    estiamtedValues = np.atleast_2d(data['r'+lungParam].to_numpy()).T
    X = np.hstack((X, estiamtedValues))
    # Return arrays to plug into sklearn's cross-validation algorithms
    return X, np.array(y)

In [6]:
X, Y = load_data('FVC', 'A', 'A', "data_FVC_N95.csv")

643 141845 441 220.0
*** Feature extraction started ***
60


686 151150 441 220.0
*** Feature extraction started ***
60


668 147396 441 220.0
*** Feature extraction started ***
60


637 140376 441 220.0
*** Feature extraction started ***
60


697 153597 441 220.0
*** Feature extraction started ***
60


...

1012 222970 441 220.0
*** Feature extraction started ***
60


1012 222970 441 220.0
*** Feature extraction started ***
60


807 177812 441 220.0
*** Feature extraction started ***
60


807 177812 441 220.0
*** Feature extraction started ***
60


807 177812 441 220.0
*** Feature extraction started ***
60


In [7]:
with open(filename, 'wb') as handle:
    pickle.dump(Audio_Features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
#testing area

cfg = tsfel.get_features_by_domain()

In [12]:
for key1 in cfg:
    #print("Key1 = ", key1)
    for key2 in cfg[key1]:
        print("\nKey2 = ", key2)
        for key3 in cfg[key1][key2]:
            if key3 == 'n_features':
                print("\tKey3 = ", cfg[key1][key2]['n_features'])


Key2 =  FFT mean coefficient
	Key3 =  nfreq

Key2 =  Fundamental frequency
	Key3 =  1

Key2 =  Human range energy
	Key3 =  1

Key2 =  LPCC
	Key3 =  n_coeff

Key2 =  MFCC
	Key3 =  num_ceps

Key2 =  Max power spectrum
	Key3 =  1

Key2 =  Maximum frequency
	Key3 =  1

Key2 =  Median frequency
	Key3 =  1

Key2 =  Power bandwidth
	Key3 =  1

Key2 =  Spectral centroid
	Key3 =  1

Key2 =  Spectral decrease
	Key3 =  1

Key2 =  Spectral distance
	Key3 =  1

Key2 =  Spectral entropy
	Key3 =  1

Key2 =  Spectral kurtosis
	Key3 =  1

Key2 =  Spectral positive turning points
	Key3 =  1

Key2 =  Spectral roll-off
	Key3 =  1

Key2 =  Spectral roll-on
	Key3 =  1

Key2 =  Spectral skewness
	Key3 =  1

Key2 =  Spectral slope
	Key3 =  1

Key2 =  Spectral spread
	Key3 =  1

Key2 =  Spectral variation
	Key3 =  1

Key2 =  Wavelet absolute mean
	Key3 =  widths

Key2 =  Wavelet energy
	Key3 =  widths

Key2 =  Wavelet entropy
	Key3 =  1

Key2 =  Wavelet standard deviation
	Key3 =  widths

Key2 =  Wavelet vari